In [1]:
import pandas as pd
import numpy as np
# from operator import itemgetter
from sentence_transformers import SentenceTransformer

In [53]:
import os 
os.listdir('../dataset_sheets/pipeline/ready/Description_new.csv')

NotADirectoryError: [Errno 20] Not a directory: '../dataset_sheets/pipeline/ready/Description_new.csv'

In [55]:
path = '../dataset_sheets/pipeline/ready/Description_new.csv'
df = pd.read_csv(path)
df['id'] = df.index
corpus = list(df['Name of the dataset'].values) #Dataset Name for some

In [56]:
# len(df)
len(corpus)

32258

In [57]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(corpus)

TypeError: 'float' object is not subscriptable

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:10]
)


array([[0.56591797, 0.67241156, 0.63447857, 0.6236371 , 0.62378097,
        0.53353137, 0.73488235, 0.5527144 , 0.6642544 ]], dtype=float32)

In [35]:
def get_top(X,rank):
    cosines = {}
    cosines_n = {}
    for x in range(X.shape[0]):
        cossims = [cosine_similarity(x, X)]
        # print('I', x)
        d = {}
        dn = {}
        for y in range(X.shape[0]):
            if x != y:
                cossim = cosine_similarity(X[x],X[y])
                d[corpus[y]] = cossim
                dn[y] = cossim
        # d = {corpus[y]:cosine_similarity(X[x],X[y])[0][0] for y in range (X.shape[0]) if x != y}
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:rank])
        cos_n_dict = dict(sorted(dn.items(), key = itemgetter(1), reverse = True)[:rank])
        cosines[x] =  '\n'.join([str(k) + ':'+ str(v) for k,v in cosdict.items()])
        cosines_n[x] =  ';'.join([str(k) + ':'+ str(v) for k,v in cos_n_dict.items()])
        print(corpus[x], cosines[x])
    return cosines, cosines_n

In [ ]:
from operator import itemgetter
sdict = {}
for x in range(sentence_embeddings.shape[0]):
        cossims = cosine_similarity([sentence_embeddings[x]],sentence_embeddings)[0]
        d = dict(zip(corpus,cossims))
        d.pop(corpus[x])
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:5])
        # cossims = [cosine_similarity(sentence_embeddings[x], sentence_embeddings)]
        # print(corpus[x])
        # print(cosdict)
        sdict[x] = '\n'.join([str(k)  for k,v in cosdict.items()]) #+ ':'+ str(v)  - add if want similarity score
print(sdict)
print(len(sdict))


In [37]:
df['similar'] = df['id'].map(sdict)


In [21]:
df['similar'].iloc[0]

'Watershed modeling and MODIS data integration\nCoastal final ecosystem goods and services (FEGS) and habitats meta-analysis data file\nGerties Creek Proper Functioning Condition Data Forms\nSCCWRP - STREAM CLASSIFICATION AND PRIORITY EXPLORER\nIntegrated Assessment Model for Valuing Water Quality Changes_BENSPLASH Input Data'

In [38]:
df.to_csv('../results/pipeline/ready/U.S. EPA Office of Research and Development (ORD)!Updated Data Categories and Use Cases-simbert-short.xlsx')